In [2]:
# =====================================================
#  🧩 Generate Mattes from Pretrained MODNet
# =====================================================
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
from pathlib import Path
import sys

In [3]:
from google.colab import drive
drive.mount('/content/mydrive', force_remount=True)

Mounted at /content/mydrive


In [4]:
# --- Paths ---
PRETRAINED_CKPT = Path('/content/mydrive/MyDrive/modnet_data/pretrained/modnet_photographic_portrait_matting.ckpt')
# INPUT_DIR  = Path('/content/mydrive/MyDrive/modnet_mattes_generate/Input_Dir')          # your input images
INPUT_DIR  = Path('/content/mydrive/MyDrive/modnet_mattes_generate/AIT_Faculty')
OUTPUT_DIR = Path('/content/mydrive/MyDrive/modnet_mattes_generate/Output_Dir')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_IMG_DIR   = Path('/content/drive/MyDrive/modnet_data/train/images')
TRAIN_MATTE_DIR = Path('/content/drive/MyDrive/modnet_data/train/mattes')
TRAIN_IMG_DIR.mkdir(parents=True, exist_ok=True)
TRAIN_MATTE_DIR.mkdir(parents=True, exist_ok=True)


In [5]:
# Clone the MODNet repository
!git clone https://github.com/ZHKKKe/MODNet.git /content/MODNet

Cloning into '/content/MODNet'...
remote: Enumerating objects: 276, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 276 (delta 21), reused 16 (delta 16), pack-reused 236 (from 1)
Receiving objects: 100% (276/276), 60.77 MiB | 28.62 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [6]:
import os, sys, importlib.util, types

# Path where you cloned the repo
repo_root = "/content/MODNet"
modnet_file = os.path.join(repo_root, "src", "models", "modnet.py")
backbones_file = os.path.join(repo_root, "src", "models", "backbones", "__init__.py")

# Create stub packages so relative imports in modnet.py work
pkg_src = types.ModuleType("src"); pkg_src.__path__ = [os.path.join(repo_root, "src")]
pkg_models = types.ModuleType("src.models"); pkg_models.__path__ = [os.path.join(repo_root, "src", "models")]
sys.modules["src"] = pkg_src
sys.modules["src.models"] = pkg_models

# Load backbones first
spec_bk = importlib.util.spec_from_file_location("src.models.backbones", backbones_file)
bk_mod = importlib.util.module_from_spec(spec_bk)
sys.modules["src.models.backbones"] = bk_mod
spec_bk.loader.exec_module(bk_mod)

# Load modnet
spec_md = importlib.util.spec_from_file_location("src.models.modnet", modnet_file)
modnet_mod = importlib.util.module_from_spec(spec_md)
sys.modules["src.models.modnet"] = modnet_mod
spec_md.loader.exec_module(modnet_mod)

# Get class
MODNet = modnet_mod.MODNet
print("✅ MODNet class loaded:", MODNet)

✅ MODNet class loaded: <class 'src.models.modnet.MODNet'>


In [7]:
# --- Device ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🧠 Using device: {device}")



🧠 Using device: cuda


In [8]:
# --- Load model ---
model = MODNet(backbone_pretrained=False).to(device)
state = torch.load(PRETRAINED_CKPT, map_location=device)
if isinstance(state, dict) and "state_dict" in state:
    state = state["state_dict"]
state = {k.replace("module.", ""): v for k, v in state.items()}
model.load_state_dict(state, strict=False)
model.eval()

print(f"✅ Loaded pretrained model: {PRETRAINED_CKPT.name}")

✅ Loaded pretrained model: modnet_photographic_portrait_matting.ckpt


In [9]:
# --- Transform for input ---
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5]),
])

In [10]:
# --- Inference function ---
def generate_matte(image_path, save_path):
    """Generate matte (alpha mask) from one image."""
    im = Image.open(image_path).convert("RGB")
    w0, h0 = im.size

    x = preprocess(im).unsqueeze(0).to(device)
    with torch.no_grad():
        _, _, matte = model(x, True)

    matte = matte[0][0].cpu().numpy()
    matte = cv2.resize(matte, (w0, h0), interpolation=cv2.INTER_LINEAR)
    matte = np.clip(matte * 255, 0, 255).astype(np.uint8)

    cv2.imwrite(str(save_path), matte)
    print(f"🖼️ Saved matte: {save_path.name}")


In [11]:
import shutil

def copy_to_train(image_path: Path, matte_path: Path):
    """Copy image and its matte to training directories."""
    dst_img = TRAIN_IMG_DIR / image_path.name
    dst_matte = TRAIN_MATTE_DIR / matte_path.name

    shutil.copy2(image_path, dst_img)
    shutil.copy2(matte_path, dst_matte)
    print(f"✅ Copied to train: {image_path.name} + {matte_path.name}")

In [13]:
valid_exts = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
generated = 0

for img_path in INPUT_DIR.glob('*'):
    if img_path.suffix in valid_exts:
        out_path = OUTPUT_DIR / f"{img_path.stem}_matte.png"
        generated += 1
        print(f"🖼️{generated} Generating matte for: {img_path.name}")
        ok = generate_matte(img_path, out_path)
        # if ok:
        #     copy_to_train(img_path, out_path)
        #     generated += 1
    else:
        print(f"⚠️ Skipped non-image: {img_path.name}")

# print(f"\n📦 Total processed and copied: {generated}")
print(f"✅ Originals → {TRAIN_IMG_DIR}")
print(f"✅ Mattes    → {TRAIN_MATTE_DIR}")

🖼️1 Generating matte for: Attaphongse Taparugssanage.jpg
🖼️ Saved matte: Attaphongse Taparugssanage_matte.png
🖼️2 Generating matte for: Rajeshwari B.jpg
🖼️ Saved matte: Rajeshwari B_matte.png
🖼️3 Generating matte for: Mrs. Yashaswi H. V.jpg
🖼️ Saved matte: Mrs. Yashaswi H. V_matte.png
🖼️4 Generating matte for: Dr. Avirut Puttiwongrak.jpg
🖼️ Saved matte: Dr. Avirut Puttiwongrak_matte.png
🖼️5 Generating matte for: Prof.Avishek Datta.jpg
🖼️ Saved matte: Prof.Avishek Datta_matte.png
🖼️6 Generating matte for: Dr. Arlene Lu-Gonzales.jpg
🖼️ Saved matte: Dr. Arlene Lu-Gonzales_matte.png
🖼️7 Generating matte for: Andrew20Macintosh_edited.jpg
🖼️ Saved matte: Andrew20Macintosh_edited_matte.png
🖼️8 Generating matte for: Prof.Ajit P.Annachhatre.jpeg
🖼️ Saved matte: Prof.Ajit P.Annachhatre_matte.png
🖼️9 Generating matte for: Dr. Amararatne Yakupitiyage.jpeg
🖼️ Saved matte: Dr. Amararatne Yakupitiyage_matte.png
🖼️10 Generating matte for: Dr. Ayushman Bhatt.jpg
🖼️ Saved matte: Dr. Ayushman Bhatt_matte

In [14]:
# =====================================================
#  🧩 Copy all images and mattes to training directories
# =====================================================
import shutil
from pathlib import Path

def copy_all_to_train(src_img_dir: Path, src_matte_dir: Path):
    """
    Copy all image–matte pairs from source directories into
    the training dataset directories.
    """
    valid_exts = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
    copied = 0

    # Ensure destination folders exist
    TRAIN_IMG_DIR.mkdir(parents=True, exist_ok=True)
    TRAIN_MATTE_DIR.mkdir(parents=True, exist_ok=True)

    # Iterate over all images in source folder
    for img_path in src_img_dir.glob('*'):
        if img_path.suffix in valid_exts:
            stem = img_path.stem
            # Find matching matte (any extension)
            matte_candidates = list(src_matte_dir.glob(stem + '*'))
            if not matte_candidates:
                print(f"⚠️ No matte found for {stem}, skipped.")
                continue

            matte_path = matte_candidates[0]
            dst_img = TRAIN_IMG_DIR / img_path.name
            dst_matte = TRAIN_MATTE_DIR / matte_path.name

            shutil.copy2(img_path, dst_img)
            shutil.copy2(matte_path, dst_matte)
            copied += 1
            print(f"✅ Copied pair: {img_path.name} + {matte_path.name}")
        else:
            print(f"⚠️ Skipped non-image: {img_path.name}")

    print(f"\n📦 Total pairs copied: {copied}")
    print(f"✅ Originals → {TRAIN_IMG_DIR}")
    print(f"✅ Mattes    → {TRAIN_MATTE_DIR}")


In [15]:
copy_all_to_train(INPUT_DIR, OUTPUT_DIR)

✅ Copied pair: Attaphongse Taparugssanage.jpg + Attaphongse Taparugssanage_matte.png
✅ Copied pair: Rajeshwari B.jpg + Rajeshwari B_matte.png
✅ Copied pair: Mrs. Yashaswi H. V.jpg + Mrs. Yashaswi H. V_matte.png
✅ Copied pair: Dr. Avirut Puttiwongrak.jpg + Dr. Avirut Puttiwongrak_matte.png
✅ Copied pair: Prof.Avishek Datta.jpg + Prof.Avishek Datta_matte.png
✅ Copied pair: Dr. Arlene Lu-Gonzales.jpg + Dr. Arlene Lu-Gonzales_matte.png
✅ Copied pair: Andrew20Macintosh_edited.jpg + Andrew20Macintosh_edited_matte.png
✅ Copied pair: Prof.Ajit P.Annachhatre.jpeg + Prof.Ajit P.Annachhatre_matte.png
✅ Copied pair: Dr. Amararatne Yakupitiyage.jpeg + Dr. Amararatne Yakupitiyage_matte.png
✅ Copied pair: Dr. Ayushman Bhatt.jpg + Dr. Ayushman Bhatt_matte.png
✅ Copied pair: Mr. Benjamin Quinlan.jpg + Mr. Benjamin Quinlan_matte.png
✅ Copied pair: Prof. Bonaventura H. W. Hadikusumo.jpg + Prof. Bonaventura H. W. Hadikusumo_matte.png
✅ Copied pair: Dr. Branesh Madhavan.jpeg + Dr. Branesh Madhavan_matte.pn

In [16]:
# =====================================================
#  🧩 Display images with filenames from path
# =====================================================
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

def show_images_from_path(folder_path: Path, limit: int = 5):
    """Display images (original or mattes) from a given folder."""
    folder_path = Path(folder_path)
    files = [f for f in folder_path.glob('*') if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
    if not files:
        print(f"⚠️ No images found in {folder_path}")
        return

    plt.figure(figsize=(15, 6))
    for i, img_path in enumerate(files[:limit]):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(1, limit, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(img_path.name, fontsize=9)
    plt.tight_layout()
    plt.show()


In [17]:
# =====================================================
#  🧩 Show a single image with its filename
# =====================================================
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

def show_image_with_name(image_path: str):
    """
    Display a single image with its filename as title.
    Works for both color and grayscale images.
    """
    image_path = Path(image_path)
    if not image_path.exists():
        print(f"❌ File not found: {image_path}")
        return

    img = cv2.imread(str(image_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"⚠️ Could not read image: {image_path}")
        return

    # Handle grayscale vs color
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)

    plt.axis('off')
    plt.title(image_path.name, fontsize=12, fontweight='bold')
    plt.show()


In [18]:
# show_image_with_name(TRAIN_MATTE_DIR / 'Dr_Mongkol_matte.png')